In [62]:
import sys
sys.path.append('../FaaSterConfig')
import subprocess
import time
import os
import select
import datetime
import re

import FaaSterConfig
sys.argv = ['./FaaSterConfig.py', '../openFaas/matmul.yml', "-c", '1', '6', '12', "-nt", "NoGPU"]

In [63]:
args, doe, stack = FaaSterConfig.main()

/Users/cameron.keenan/.pyenv/versions/FaaSterConfig/lib/python3.11/site-packages/doepy/doe_functions.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NoGPU' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.iloc[i][j]=r[j][int(df.iloc[i][j])]


In [64]:
doe

,CPU,Mem,NodeType,i,NodeTypeStr,funcName
0,1.0,248.0,0.0,1,NoGPU,matmul_CPU1.0_Mem_248.0_NoGPU
1,6.0,248.0,0.0,2,NoGPU,matmul_CPU6.0_Mem_248.0_NoGPU
2,12.0,248.0,0.0,3,NoGPU,matmul_CPU12.0_Mem_248.0_NoGPU
3,1.0,1024.0,0.0,4,NoGPU,matmul_CPU1.0_Mem_1024.0_NoGPU
4,6.0,1024.0,0.0,5,NoGPU,matmul_CPU6.0_Mem_1024.0_NoGPU
5,12.0,1024.0,0.0,6,NoGPU,matmul_CPU12.0_Mem_1024.0_NoGPU
6,1.0,16384.0,0.0,7,NoGPU,matmul_CPU1.0_Mem_16384.0_NoGPU
7,6.0,16384.0,0.0,8,NoGPU,matmul_CPU6.0_Mem_16384.0_NoGPU
8,12.0,16384.0,0.0,9,NoGPU,matmul_CPU12.0_Mem_16384.0_NoGPU


In [65]:

def read_line_with_timeout(pipe, timeout):
    rlist, _, _ = select.select([pipe], [], [], timeout)
    if rlist:
        return pipe.readline()
    else:
        return None

def run_function(funcName, data="100", stackYml=None, port=4500, MAXBUF=120, verbose=False):
    if not stackYml: 
        # Assume stack.yml file is same name as function
        stackYml = f"../openFaas/{funcName}.yml"

    logFile = f"/tmp/openFaas/{funcName}.log"
    os.makedirs(os.path.dirname(logFile), exist_ok=True)

    cmd = f"cd {os.path.dirname(stackYml)}; faas-cli local-run {funcName} -f {stackYml} -p {port} --quiet 2>&1 | tee {logFile}"
    if verbose:
        print("running command: ")
        print(cmd)
    process = subprocess.Popen(cmd, bufsize=MAXBUF, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    start = datetime.datetime.now()
    duration = None
#     while (datetime.datetime.now() - start).total_seconds() < 15:
    for line in process.stdout:
#     while True:
#         line = read_line_with_timeout(process.stdout, .1)
        if line:
            if verbose:
                print(line)
            if re.search("Listening on port: 8080", line):
                # TODO use requests and send payload from input 
                os.system(f"curl http://0.0.0.0:{port} -X POST  --data '{data}' >/dev/null 2>&1")
            elif (match:= re.search("Duration: ((?:\d|.)+)s", line)):
                duration = float(match.groups()[0])
                break
                
    if not process.poll():
        process.kill()
    # Ensure all subprocesses are killed for this function
    os.system("""kill -9 $(ps -ef | grep -e "faas-cli local-run %s" -e "docker run --name %s" -e "run.*%s" | awk '{print $2}')""" % (funcName,funcName,funcName))

    return duration

In [66]:
# duration = run_function(doe.iloc[0].funcName, "../openFaas/stack.yml", port=4501, verbose=True)
# print(f"result: {duration}")

In [67]:
def getTime(row):
    return run_function(row.funcName, data="1000", stackYml="../openFaas/matmul_gen.yml", port=(4500+row.i), verbose=False)
    
doe = FaaSterConfig.applyThreaded(doe, getTime, max_workers=4)

In [68]:
doe.sort_values(by=["time"]) 

,CPU,Mem,NodeType,i,NodeTypeStr,funcName,time
7,6.0,16384.0,0.0,8,NoGPU,matmul_CPU6.0_Mem_16384.0_NoGPU,0.657340
8,12.0,16384.0,0.0,9,NoGPU,matmul_CPU12.0_Mem_16384.0_NoGPU,0.691017
2,12.0,248.0,0.0,3,NoGPU,matmul_CPU12.0_Mem_248.0_NoGPU,0.739137
4,6.0,1024.0,0.0,5,NoGPU,matmul_CPU6.0_Mem_1024.0_NoGPU,0.915169
5,12.0,1024.0,0.0,6,NoGPU,matmul_CPU12.0_Mem_1024.0_NoGPU,1.226768
6,1.0,16384.0,0.0,7,NoGPU,matmul_CPU1.0_Mem_16384.0_NoGPU,1.281898
1,6.0,248.0,0.0,2,NoGPU,matmul_CPU6.0_Mem_248.0_NoGPU,1.400259
0,1.0,248.0,0.0,1,NoGPU,matmul_CPU1.0_Mem_248.0_NoGPU,1.434297
3,1.0,1024.0,0.0,4,NoGPU,matmul_CPU1.0_Mem_1024.0_NoGPU,2.294837
